In [1]:
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from Parser import parse_frames
from scipy.signal import medfilt
from Functions import analyse_each_rep, find_extremas
from JointAngles import JointAngles
import numpy as np
import glob
import os
import pickle
import pandas as pd
from functools import reduce


In [10]:
# input_folder = "C:\\Users\\altaok\\Documents\\GitHub\\IndividualProject\\keypoints_for_all\\shoulder press"
input_folder = 'C:\\Users\\ak5u16\\Desktop\\IndividualProject\\keypoints_for_all\\shoulder press'
folder_paths = glob.glob(os.path.join(input_folder, 'shoulder_press*'))
points_folder_name = os.path.basename(input_folder)


def get_data_for_dataset(folder_paths, points_folder_name):
    angle_arrays = []
    for folder in folder_paths:
        video_name = os.path.basename(folder)
        label = 0 if 'correct' in folder else 1
        frame_poses = parse_frames(folder)
        joint_angles = JointAngles(points_folder_name, frame_poses)
        
        if points_folder_name == 'shoulder press':  
            left_upArm_forearm_angles = np.array(joint_angles.left_upArm_forearm_angles)
            left_upArm_forearm_angles = np.nan_to_num(left_upArm_forearm_angles)
            left_upArm_forearm_angles_filtered = medfilt(medfilt(left_upArm_forearm_angles, 5), 5)
            
            right_upArm_forearm_angles = np.array(joint_angles.right_upArm_forearm_angles)
            right_upArm_forearm_angles = np.nan_to_num(right_upArm_forearm_angles)
            right_upArm_forearm_angles_filtered = medfilt(medfilt(right_upArm_forearm_angles, 5), 5)
            
            left_upArm_trunk_angles = np.array(joint_angles.left_upArm_trunk_angles)
            left_upArm_trunk_angles = np.nan_to_num(left_upArm_trunk_angles)
            left_upArm_trunk_angles_filtered = medfilt(medfilt(left_upArm_trunk_angles, 5), 5)
            
            right_upArm_trunk_angles = np.array(joint_angles.right_upArm_trunk_angles)
            right_upArm_trunk_angles = np.nan_to_num(right_upArm_trunk_angles)
            right_upArm_trunk_angles_filtered = medfilt(medfilt(right_upArm_trunk_angles, 5), 5)
            
            extremas1 = filter_extremas(left_upArm_trunk_angles_filtered, find_extremas(left_upArm_trunk_angles_filtered, maxima=False), maxima=False)
            extremas2 = filter_extremas(right_upArm_trunk_angles_filtered, find_extremas(right_upArm_trunk_angles_filtered, maxima=False), maxima=False)
            
            angle_arrays.append((label, extremas1, extremas2, [left_upArm_forearm_angles, right_upArm_forearm_angles, left_upArm_trunk_angles, right_upArm_trunk_angles]))

           
        else: 
            print('Error: Wrong folder path! Has to be: shoulder press')
      
        return angle_arrays
                                             
                                    

def fill_dataframe(angle_arrays, exercise_folder_name):
    df_list = []
    
    print('Filling dataset with ' + str(exercise_folder_name) + ' data...')
   
    for tup in angle_arrays:
        label = tup[0]
        extremas1 = tup[1]
        extremas2 = tup[2]
        uf_angles1, uf_angles2, ut_angles1, ut_angles2 = tup[3]
        # Extract rep angles
        each_side_rep_angles = analyse_each_rep('shoulder press', 'dataset', extremas1=extremas1, uf_angles1=uf_angles1, ut_angles1=ut_angles1, extremas2=extremas2, uf_angles2=uf_angles2, ut_angles2=ut_angles2)
       
        s1 = pd.Series(each_rep_angles, name='Angle_array')
        s2 = pd.Series([label for n in range(len(each_rep_angles))], name='Label')
        df = pd.concat([s1,s2], axis=1)
        df_list.append(df)

    return pd.concat(df_list).reset_index(drop=True)
        

In [11]:
angle_arrays = get_data_for_dataset(folder_paths, points_folder_name) 
df = fill_dataframe(angle_arrays, points_folder_name)
print(df.info())

Filling dataset with shoulder press data...
Error: Rep counts for left and right arms are not equal


AttributeError: 'tuple' object has no attribute 'size'